In [70]:
import pandas as pd
votes = pd.read_csv('Hall_votes.csv')
votes = votes.drop(columns = ['chamber', 'prob'])
castCode = []
for line in votes.cast_code:
    if line >= 7:
        castCode.append(-1)
    elif line > 3:
        castCode.append(1)
    else:
        castCode.append(0)
votes.cast_code = castCode
congNum = 1
congList = []
while(congNum < 117):
    congList.append((votes[votes['congress'] == congNum]).drop(columns = 'congress'))
    congNum += 1
finalVotes = []
for congress in congList:
    congress = congress.pivot_table(index = 'icpsr', columns = 'rollnumber', fill_value=2)
    congress = congress.reset_index()
#     congress['icpsr'].astype(int)
    finalVotes.append((congress.values.astype(str).tolist()))
for vote in finalVotes:
    for each in vote:
        index= 1
        length = len(each)
        while(index < length):
            if each[index] == '2':
                each[index] = 'None'
            index +=1

In [2]:
party = pd.read_csv('Hall_members.csv')
party = party.drop(columns = ['chamber', 'state_icpsr', 'district_code', 'state_abbrev', 'occupancy', 'last_means', 'bioname',
                             'bioguide_id', 'born', 'died','conditional','nokken_poole_dim1', 'nokken_poole_dim2', 
                            'nominate_dim1', 'nominate_dim2', 'nominate_number_of_errors', 'nominate_log_likelihood','nominate_geo_mean_probability',
                             'nominate_number_of_votes'])

party = party.set_index('congress')

congList = []
congNum = 1
while(congNum < 117):
    newCong = party.loc[congNum]
    newCong = newCong.sort_values(by=['icpsr'])
    newCong = newCong.reset_index(drop=True)
    congList.append(newCong)
    congNum += 1
for cong in congList:
    newPartyList= []
    parties = cong['party_code'].value_counts().index.tolist()
    for each in cong['party_code']:
        if each == parties[0]:
            newPartyList.append(0)
        elif each == parties[1]:
            newPartyList.append(1)
        else:
            newPartyList.append(-1)
    cong.party_code = newPartyList

finalParty = []
for cong in congList:
    finalParty.append(cong.values.tolist())

In [3]:
def dropMissing(congVote, congParty):
    LVote = len(congVote)
    PVote = len(congParty)
    if(LVote == PVote):
        return congVote, congParty
    print(LVote)
    print(PVote)
    maxC = LVote if LVote > PVote else PVote
    z = 0
    while(z < maxC):
        if int(congVote[z][0]) != congParty[z][0]:
            if LVote < PVote:
                print("dropping")
                print(congParty[z])
                del congParty[z]
                z-=1
                maxC -= 1
            elif PVoe > LVote:
                print("dropping")
                print(congVote[z])
                del congVote[z]
                z-=1
                macC -= 1
            else:
                print("Same length different values")
        z+=1
    return congVote, congParty

In [4]:
import copy
def getVote(num):
    votes = copy.deepcopy(finalVotes)
    return votes[num - 1]

In [5]:
def getParty(num):
    partys = finalParty[:]
    return partys[num - 1]

In [6]:
def checkAccuracy(trueParty, testParty):
    index = 0
    count = len(trueParty)
    trueCount = 0
    while(index < count):
        if trueParty[index] == testParty[index]:
            trueCount += 1
        index += 1
    accuracy = trueCount/count
    return accuracy

In [21]:
def getClusters(x, dissim_func):
    kmodes_cao = KModes(n_clusters=2, init='Cao', cat_dissim=dissim_func, verbose=0)
    kmodes_cao.fit_predict(x)

#   Print cluster centroids of the trained model.
#    print('k-modes (Cao) centroids:')
#    print(kmodes_cao.cluster_centroids_)
    
#   Print training statistics
    print('Final training cost: {}'.format(kmodes_cao.cost_))
    print('Training iterations: {}'.format(kmodes_cao.n_iter_))
#    print(kmodes_cao.labels_)
    print('Num reps           : {}'.format(len(kmodes_cao.labels_)))
    return kmodes_cao

In [22]:
def testClusters(clusters, icpsrList, clustMap):
    
    clustMapCopy = copy.deepcopy(clustMap)
    cLen = len(clustMapCopy)
    ind = 0
    while(ind < cLen):
        if clustMapCopy[ind] == 0:
            clustMapCopy[ind] = 1
        else:
            clustMapCopy[ind] = 0
        ind+=1
    index = 0
    testParty = [] #first clust
    testParty2 = []
    while(index < len(icpsrList)):
        mem = icpsrList[index]
        c = clustMap[index]
        d = clustMapCopy[index]
        testParty.append([mem, c])
        testParty2.append([mem, d])
        index +=1
    return testParty, testParty2

In [29]:
def matching_dissim(a, b, **_):
    """Simple matching dissimilarity function"""
    return np.sum(a != b, axis=1)

In [65]:
def ignore_abst_dissim(a, b, **_):
    """Simple matching dissimilarity function"""
    abst = np.full_like(a, -1)
    return np.sum(np.logical_or(np.logical_or(a!=abst, b!=abst),a!=b), axis=1)  # always dissim if either is abstention

In [50]:
def half_abst_dissim(a, b, **_):
    """Simple matching dissimilarity function"""
    return np.sum(a!=b if np.logical_or(a==-1, b==-1) else .5, axis=1)  # always dissim if either is abstention

In [76]:
def run_kmodes_single_cong(dissim_func):
    print('num votes:           {}'.format(len(aVote[0])))
    kmodesCao = getClusters(x, dissim_func)
    clusters = kmodesCao.cluster_centroids_
    print('partisanship       : {}'.format(cluster_distance(clusters)/len(aVote[0])))
    clustMap = kmodesCao.labels_    
    testParty, testParty2 = testClusters(clusters, icpsrList, clustMap)
    acc = checkAccuracy(aParty, testParty)
    acc2 = checkAccuracy(aParty, testParty2)
    print('acc:                 {}'.format(acc if acc > acc2 else acc2))

In [46]:
def cluster_distance(clusters):
    return np.sum(clusters[0] != clusters[1], axis=0)

In [77]:
from kmodes.kmodes import KModes
import numpy as np
import difflib


loop =  115 #len(finalVotes)
i = 1
while(i < loop):
    
    print('***CONGRESS NUM: {}'.format(i))
    
    aParty = getParty(i)
    aVote = getVote(i)
    aVote, aParty = dropMissing(aVote, aParty)
    
    icpsrList = []
    for each in aVote:
        icpsrList.append(int(each.pop(0))) #seperate icpsr from votes cast into icpsrList and x
    x = np.array(aVote)
    print("MATCHING DISSIM")
    run_kmodes_single_cong(matching_dissim)
    #print("ZERO ABST DISSIM")
    #run_kmodes_single_cong(ignore_abst_dissim)
    i+=1

***CONGRESS NUM: 1
MATCHING DISSIM
num votes:           109
Final training cost: 3222.0
Training iterations: 2
Num reps           : 67
partisanship       : 0.7155963302752294
acc:                 0.6119402985074627
***CONGRESS NUM: 2
MATCHING DISSIM
num votes:           102
Final training cost: 2899.0
Training iterations: 2
Num reps           : 72
partisanship       : 0.803921568627451
acc:                 0.7916666666666666
***CONGRESS NUM: 3
MATCHING DISSIM
num votes:           69
Final training cost: 2945.0
Training iterations: 2
Num reps           : 111
partisanship       : 0.855072463768116
acc:                 0.8558558558558559
***CONGRESS NUM: 4
MATCHING DISSIM
num votes:           83
Final training cost: 3831.0
Training iterations: 2
Num reps           : 116
partisanship       : 0.6987951807228916
acc:                 0.9224137931034483
***CONGRESS NUM: 5
MATCHING DISSIM
num votes:           155
Final training cost: 6189.0
Training iterations: 2
Num reps           : 118
partis

MATCHING DISSIM
num votes:           613
Final training cost: 56380.0
Training iterations: 2
Num reps           : 201
partisanship       : 0.7862969004893964
acc:                 0.8656716417910447
***CONGRESS NUM: 40
MATCHING DISSIM
num votes:           717
Final training cost: 77672.0
Training iterations: 5
Num reps           : 234
partisanship       : 0.800557880055788
acc:                 0.6495726495726496
***CONGRESS NUM: 41
MATCHING DISSIM
num votes:           634
Final training cost: 78061.0
Training iterations: 2
Num reps           : 257
partisanship       : 0.8012618296529969
acc:                 0.9688715953307393
***CONGRESS NUM: 42
MATCHING DISSIM
num votes:           517
Final training cost: 58508.0
Training iterations: 2
Num reps           : 255
partisanship       : 0.7350096711798839
acc:                 0.9725490196078431
***CONGRESS NUM: 43
MATCHING DISSIM
num votes:           475
Final training cost: 59922.0
Training iterations: 3
Num reps           : 305
partisanshi

MATCHING DISSIM
num votes:           152
Final training cost: 24401.0
Training iterations: 3
Num reps           : 455
partisanship       : 0.4868421052631579
acc:                 0.9714285714285714
***CONGRESS NUM: 78
MATCHING DISSIM
num votes:           156
Final training cost: 23504.0
Training iterations: 3
Num reps           : 452
partisanship       : 0.5192307692307693
acc:                 0.9513274336283186
***CONGRESS NUM: 79
MATCHING DISSIM
num votes:           231
Final training cost: 37396.0
Training iterations: 3
Num reps           : 449
partisanship       : 0.5800865800865801
acc:                 0.933184855233853
***CONGRESS NUM: 80
MATCHING DISSIM
num votes:           163
Final training cost: 20143.0
Training iterations: 3
Num reps           : 450
partisanship       : 0.4662576687116564
acc:                 0.9844444444444445
***CONGRESS NUM: 81
MATCHING DISSIM
num votes:           275
Final training cost: 39310.0
Training iterations: 3
Num reps           : 446
partisanshi